# Importing my own Books from Goodreads export tool
Goodreads export using: https://www.goodreads.com/review/import


In [2]:
# set working directory to same place ass app.py to import programs the same way as the app
import os
current_directory = os.getcwd()
if 'notebooks' in current_directory:
    parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
    os.chdir(parent_directory)
os.getcwd()

'/Users/elisealstad/code/mybook-dashboard'

In [3]:
# Improt packages
import pandas as pd
import json 
import numpy as np

# Import functions from apps folder
# from apps.collect_data import *

pd.set_option('max_colwidth', 50)
pd.set_option('display.max_columns', 80)

In [6]:
mybooksgr = pd.read_csv("assets/goodreads_library_export.csv")
# mybooksgr = mybooksgr.rename(columns=lambda x: x.replace(' ', '_'))
mybooksgr = mybooksgr.sample(250)

In [7]:
len(mybooksgr)

250

In [8]:
import nest_asyncio
nest_asyncio.apply()

from apps.async_googleapi import book_info_add, asyncio
from apps.api import api_key
pimydf = asyncio.run(book_info_add(mybooksgr, api_key))

Black Mountain
The Fox Wife
Why We Sleep: Unlocking the Power of Sleep and Dreams
Verity
1Q84 (1Q84, #3)
The Guest Cat
All the Dangerous Things
Yinka, Where Is Your Huzband?
Home Fire
Nullpunkt (Alexander Blix & Emma Ramm, #1)
A Feast for Crows (A Song of Ice and Fire, #4)
Icebreaker (UCMH, #1)
Courage To Be Disliked / The Road Back to You / Mindset
After Dark
Yellowface
Carrie Soto Is Back
Berlinerpoplene (Familien Neshov, #1)
The Art of Statistics: How to Learn from Data
The Unhoneymooners (Unhoneymooners, #1)
The Day I Die: The Untold Story of Assisted Dying in America
Strange Weather in Tokyo
Everyone in My Family Has Killed Someone (Ernest Cunningham, #1)
It Ends with Us (It Ends with Us, #1)
The Evening and the Morning (Kingsbridge, #0)
Beautiful Malice
Pachinko
Before the Coffee Gets Cold
Sweet Bean Paste
Assistant to the Villain (Assistant to the Villain, #1)
How to Lie with Statistics
May We Be Forgiven
Unruly: A History of England's Kings and Queens
And Then There Were None
H

# Merge dataframes 
mybooks = pd.merge(mybooksgr,
                     pimydf,
                     on='Title', 
                     suffixes = ('_Goodreads', '_GoogleBooks'), 
                     how='left')

In [9]:

from apps.dataimport import dataprep
nmyreads = dataprep(mybooksgr, pimydf)

In [10]:
import pickle
nmyreads.to_pickle("assets/my_books.pkl")

# Data cleaning books steps, PS this is already run above
- page count categories
- filter if book is read of not 

In [35]:
# Page count category variable

def categorize_pages(number_of_pages):
    if number_of_pages >= 100 and number_of_pages <= 249:
        return '100-249'
    elif number_of_pages >= 250 and number_of_pages <= 349:
        return '250-349'
    elif number_of_pages >= 350 and number_of_pages <= 449:
        return '350-449'
    elif number_of_pages >= 450 and number_of_pages <= 599:
        return '450-599'
    elif number_of_pages >= 600 and number_of_pages <= 749:
        return '600-749'
    elif number_of_pages >= 750 and number_of_pages <= 999:
        return '750-999'
    else:
        return '1000+'

# Apply the categorize_pages function to create the 'Page_Cat' column
mybooks['Page_Cat'] = mybooks['Number_of_Pages'].apply(categorize_pages)

# Define the desired order of categories
category_order = ['100-249', '250-349', '350-449', '450-599', '600-749', '750-999', '1000+']

# Convert the 'Page_Cat' column to a categorical variable with the specified order
mybooks['Page_Cat'] = pd.Categorical(mybooks['Page_Cat'], categories=category_order, ordered=True)


In [36]:
# drop duplicates
mybooks = mybooks.drop_duplicates(subset=['Title', 'Author'])

In [38]:
# Create year and quarter read variable 
#  Impute data_added where date_read  is na
mybooks['Date_Read'] = np.where(mybooks['Date_Read'].isnull() & mybooks['Read_Count']==1, mybooks['Date_Added'], mybooks['Date_Read'])

# Convert 'Date_Read' column to datetime type
mybooks['Date_Read'] = pd.to_datetime(mybooks['Date_Read'], format='mixed')

# Extract year and quarter from 'Date_Read' column
mybooks['Year'] = mybooks['Date_Read'].dt.year
mybooks['Quarter'] = mybooks['Date_Read'].dt.quarter

# Create a new column combining year and quarter
mybooks['Year_Quarter'] = np.where(mybooks['Date_Read'].notnull(), mybooks['Year'].astype(str) + '-Q' + mybooks['Quarter'].astype(str), np.nan)
# Replace '.0' in the Year_Quarter column with an empty string
mybooks['Year_Quarter'] = mybooks['Year_Quarter'].fillna('').str.replace('.0', '')

# Convert Year_Quarter to categorical variable
mybooks['Year_Quarter'] = pd.Categorical(mybooks['Year_Quarter'], ordered=True)

In [41]:
# filter na in publication year and make column publication year integer 
mybooks['Original_Publication_Year'] = mybooks['Original_Publication_Year'].fillna( 0)
mybooks['Original_Publication_Year'] = mybooks['Original_Publication_Year'].astype(int)

In [43]:
# Making sure all na is set as np.nan and not as a string variable (had this issue with one variable)
import numpy as np
mybooks = mybooks.replace('nan', np.nan)
mybooks = mybooks.replace('NaN', np.nan)

# Collect book topics 

In [11]:
# Collect topics for my own books from OLapi
from apps.collect_data import get_book_topics
my_topics = get_book_topics(mybooksgr)

In [12]:
dict = mybooksgr[['Author','Title']].to_dict()
list(set(dict['Title'].values()))
    

["Don't Let Her Stay",
 'Tung tids tale',
 'Lucifers evangelium (Bjørn Beltø, #3)',
 'Penance',
 'A Room with a View',
 'After Dark',
 'An American Marriage',
 'Snømannen (Harry Hole, #7)',
 'Icebreaker (UCMH, #1)',
 'Girl, Woman, Other',
 'Norwegian Wood',
 'Neverwhere (London Below, #1)',
 'Malibu Rising',
 'How to Lie with Statistics',
 'The Japanese Lover',
 '21 Lessons for the 21st Century',
 "The Signal and the Noise: Why So Many Predictions Fail—But Some Don't",
 'The History of Bees (Klimakvartetten, #1)',
 'Shatter Me (Shatter Me, #1)',
 'The Charity Shop Detective Agency (The Charity Shop Detective Agency, #1)',
 'Then She Vanishes',
 'My Dark Vanessa',
 'The Silent Patient',
 'One Last Secret',
 "Unruly: A History of England's Kings and Queens",
 'Sweet Damage',
 'Sweet Bean Paste',
 'Normal People',
 'Behave: The Biology of Humans at Our Best and Worst',
 'Strange Weather in Tokyo',
 'Looking for Bono (Twenty in 2020)',
 'The Kind Worth Killing (Henry Kimball/Lily Kintner, 

In [13]:
df = pd.DataFrame(dict)
df

,Author,Title
234,Simon Bestwick,Black Mountain
56,Yangsze Choo,The Fox Wife
434,Matthew Walker,Why We Sleep: Unlocking the Power of Sleep and...
306,Colleen Hoover,Verity
258,Haruki Murakami,"1Q84 (1Q84, #3)"
...,...,...
201,Eliza Clark,Penance
440,Roy Jacobsen,The Unseen (The Barrøy Chronicles #1)
247,Jessica George,Maame
70,Michael Cragg,"Reach for the Stars: 1996–2006: Fame, Fallout ..."


In [15]:
# Writes the topics as 
with open("assets/my_topics.json", "w") as outfile:
    json.dump(my_topics, outfile)

In [1]:
mybooksgr.query('`Read Count` ==1').head(2).to_csv("assets/my_books_test.csv", index=False)

NameError: name 'mybooksgr' is not defined